In [1]:
import os
from glob import glob
import subprocess

epsg = "4326" 

In [8]:

def reproject_raster(input_raster, output_raster, res=0.0001111111111111111164, 
                     epsg_code="4749", resampling_method="near", overwrite=True):
   
    command = [
        "gdalwarp",
        "-t_srs", f"EPSG:{epsg_code}",
        "-r", resampling_method,
        "-tr", str(res), str(res),
    ]
    
    if overwrite:
        command.append("-overwrite")
    
    command.extend([input_raster, output_raster])

    try:
        subprocess.run(command, check=True)
        print(f"[SUCCESS] Reprojection completed: {output_raster}")
    except subprocess.CalledProcessError as e:
        print(f"[ERROR] Reprojection failed: {e}")
    except FileNotFoundError:
        print("[ERROR] 'gdalwarp' command not found. Make sure GDAL is installed and in your PATH.")



In [ ]:
tls_fn = "/media/ljp238/12TBWolf/ARCHIEVE/LIDAR_DTM/data/AngorW/TLSLidarDEM/AngkorDTM50cm.tif"
mkd_fn = "/media/ljp238/12TBWolf/ARCHIEVE/LIDAR_DTM/data/Mekong/delta_s_m2070.tif"
rgn_files = glob("/media/ljp238/12TBWolf/ARCHIEVE/LIDAR_DTM/data/RNG_Transects/*.tif")
outdir4326 = "/media/ljp238/12TBWolf/ARCHIEVE/LIDAR_DTM/reproj4326" #
outdir4749 =  "/media/ljp238/12TBWolf/ARCHIEVE/LIDAR_DTM/reproj4749"  

In [10]:
import os
from concurrent.futures import ProcessPoolExecutor, as_completed

def process_file(fi, outdir, epsg):
    os.makedirs(outdir, exist_ok=True)
    fn = os.path.basename(fi)
    out_fn = os.path.join(outdir, fn)
    if not os.path.exists(out_fn):
        reproject_raster(fi, out_fn, epsg_code=epsg)
    else:
        print(f"File already exists: {out_fn}")

def parallel_process(tif_files, outdir, epsg):
    with ProcessPoolExecutor() as executor:
        futures = [executor.submit(process_file, fi, outdir, epsg) for fi in tif_files]
        for future in as_completed(futures):
            future.result()  # Optional, to catch exceptions if any



In [11]:
tif_files = [tls_fn, mkd_fn] + rgn_files

In [ ]:
parallel_process(tif_files=tif_files, outdir=outdir4326, epsg="4326")

In [ ]:

parallel_process(tif_files=tif_files, outdir=outdir4749, epsg="4749")